### 1. 데이터 불러오기

In [3]:
# 라이브러리 정의 import tensorflow.keras
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import pandas as pd

C:\Users\User\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\User\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [4]:
featuresData = pd.read_csv("https://raw.githubusercontent.com/hyokwan/python-lecture/master/dataset/cars.csv")

In [5]:
featuresData.head()

,age,gender,miles,debt,income,sales
0,28,0,23,0,4099,620
1,26,0,27,0,2677,1792
2,30,1,58,41576,6215,27754
3,26,1,25,43172,7626,28256
4,20,1,17,6979,8071,4438


### 2. 특성선정 및 데이터 분리

In [6]:
label = ['sales']
corrdf = featuresData.corr()

In [7]:
features = list(corrdf[(abs(corrdf.sales) > 0.5) & (corrdf.sales <1)].index)
features

['miles', 'debt', 'income']

In [8]:
from sklearn.model_selection import train_test_split

C:\Users\User\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [9]:
trainingData_features,testData_features,trainingData_label,testData_label,trainingData_all,testData_all = \
train_test_split(featuresData[features],featuresData[label],featuresData, test_size=0.2, random_state=1)

In [10]:
def norm(x):
    return (x/x.max())

trainingData_features_normed = norm(trainingData_features)
testData_features_normed = norm(testData_features)

### 3. 모델 생성

In [11]:
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn import tree
from sklearn import linear_model
from sklearn import ensemble
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn import neighbors

In [12]:
inputDim = trainingData_features_normed.loc[0,:].shape
inputDim

(3,)

In [13]:
model_method_svm = svm.SVC(random_state=1)
model_method_dt = tree.DecisionTreeClassifier(random_state =1)    #DecisionTreeRegressor(random_state =1)
model_method_kn = neighbors.KNeighborsClassifier(n_neighbors=10)
model_method_rf = ensemble.RandomForestClassifier(random_state =1, n_estimators= 10)

In [14]:
model = Sequential()
model.add(Dense(units=8, activation="relu",input_shape=inputDim))
model.add(Dense(units=8, activation="relu"))
model.add(Dense(units=1, activation="relu"))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 8)                 32        
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 72        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 9         
Total params: 113
Trainable params: 113
Non-trainable params: 0
_________________________________________________________________


### 4. 모델 컴파일(훈련준비)

In [15]:
model.compile(loss='mean_squared_error', optimizer = 'adam', metrics=['mean_squared_error','mean_absolute_error'])

### 5. 모델 훈련(callback 정의)

In [16]:
#callback List를 만들기 위한 선언 및 변수 설정
from keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
from datetime import datetime
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
from datetime import datetime
import os

log_folder = datetime.now().strftime("%Y%m%d_%H%M%S")



log_dirs = os.path.join("logs",log_folder)

Using TensorFlow backend.


In [17]:
callback_list = [
    TensorBoard(log_dir = log_dirs),
    EarlyStopping(monitor="val_loss", patience=10), 
    ModelCheckpoint(filepath="./model_r_weights.h5", monitor="val_loss", verbose=1, save_best_only=True)   
]

In [18]:
model_svm = model_method_svm.fit(trainingData_features_normed ,trainingData_label)
model_dt  = model_method_dt.fit (trainingData_features_normed ,trainingData_label)
model_kn  = model_method_kn.fit (trainingData_features_normed ,trainingData_label)
model_rf  = model_method_rf.fit (trainingData_features_normed ,trainingData_label)

C:\Users\User\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a

In [19]:
model.fit(x=trainingData_features_normed,
          y=trainingData_label,
          batch_size=32,
          epochs=2000,
          validation_split=0.2,
          callbacks=callback_list)

W1015 17:50:44.025910 12784 training.py:504] Falling back from v2 loop because of error: Failed to find data adapter that can handle input: <class 'pandas.core.frame.DataFrame'>, <class 'NoneType'>


Train on 616 samples, validate on 154 samples
Epoch 1/2000
352/616 [================>.............] - ETA: 0s - loss: 236839482.1818 - mean_squared_error: 236839488.0000 - mean_absolute_error: 12320.8467
Epoch 00001: val_loss improved from inf to 187157899.22078, saving model to ./model_r_weights.h5
616/616 [==============================] - 0s 306us/sample - loss: 219885906.2857 - mean_squared_error: 219885920.0000 - mean_absolute_error: 11734.9033 - val_loss: 187157899.2208 - val_mean_squared_error: 187157904.0000 - val_mean_absolute_error: 10915.1123
Epoch 2/2000
 32/616 [>.............................] - ETA: 0s - loss: 224204896.0000 - mean_squared_error: 224204896.0000 - mean_absolute_error: 11974.3301
Epoch 00002: val_loss improved from 187157899.22078 to 187155429.61039, saving model to ./model_r_weights.h5
616/616 [==============================] - 0s 79us/sample - loss: 219883399.8961 - mean_squared_error: 219883408.0000 - mean_absolute_error: 11734.8086 - val_loss: 187155429

Epoch 17/2000
 32/616 [>.............................] - ETA: 0s - loss: 128111024.0000 - mean_squared_error: 128111024.0000 - mean_absolute_error: 9422.2734
Epoch 00017: val_loss improved from 186670702.33766 to 186577482.18182, saving model to ./model_r_weights.h5
616/616 [==============================] - 0s 63us/sample - loss: 219286431.1688 - mean_squared_error: 219286432.0000 - mean_absolute_error: 11712.9023 - val_loss: 186577482.1818 - val_mean_squared_error: 186577488.0000 - val_mean_absolute_error: 10891.7764
Epoch 18/2000
 32/616 [>.............................] - ETA: 0s - loss: 122781680.0000 - mean_squared_error: 122781680.0000 - mean_absolute_error: 7923.2158
Epoch 00018: val_loss improved from 186577482.18182 to 186474054.64935, saving model to ./model_r_weights.h5
616/616 [==============================] - 0s 62us/sample - loss: 219176184.6234 - mean_squared_error: 219176192.0000 - mean_absolute_error: 11708.9102 - val_loss: 186474054.6494 - val_mean_squared_error: 186

Epoch 33/2000
 32/616 [>.............................] - ETA: 0s - loss: 202692032.0000 - mean_squared_error: 202692032.0000 - mean_absolute_error: 10769.3750
Epoch 00033: val_loss improved from 183721990.85714 to 183408034.07792, saving model to ./model_r_weights.h5
616/616 [==============================] - 0s 70us/sample - loss: 215830046.7532 - mean_squared_error: 215830016.0000 - mean_absolute_error: 11588.0908 - val_loss: 183408034.0779 - val_mean_squared_error: 183408048.0000 - val_mean_absolute_error: 10766.2422
Epoch 34/2000
 32/616 [>.............................] - ETA: 0s - loss: 112009144.0000 - mean_squared_error: 112009144.0000 - mean_absolute_error: 8459.9961
Epoch 00034: val_loss improved from 183408034.07792 to 183084845.92208, saving model to ./model_r_weights.h5
616/616 [==============================] - 0s 57us/sample - loss: 215467241.8701 - mean_squared_error: 215467248.0000 - mean_absolute_error: 11575.1035 - val_loss: 183084845.9221 - val_mean_squared_error: 18

Epoch 49/2000
 32/616 [>.............................] - ETA: 0s - loss: 229394976.0000 - mean_squared_error: 229394976.0000 - mean_absolute_error: 12424.7617
Epoch 00049: val_loss improved from 176690206.96104 to 176090155.42857, saving model to ./model_r_weights.h5
616/616 [==============================] - 0s 63us/sample - loss: 207705992.7273 - mean_squared_error: 207706000.0000 - mean_absolute_error: 11291.1162 - val_loss: 176090155.4286 - val_mean_squared_error: 176090160.0000 - val_mean_absolute_error: 10472.4512
Epoch 50/2000
 32/616 [>.............................] - ETA: 0s - loss: 192289920.0000 - mean_squared_error: 192289920.0000 - mean_absolute_error: 10981.8994
Epoch 00050: val_loss improved from 176090155.42857 to 175473433.55844, saving model to ./model_r_weights.h5
616/616 [==============================] - 0s 63us/sample - loss: 207032954.5974 - mean_squared_error: 207032928.0000 - mean_absolute_error: 11265.7275 - val_loss: 175473433.5584 - val_mean_squared_error: 1

...

Epoch 609/2000
 32/616 [>.............................] - ETA: 0s - loss: 11821976.0000 - mean_squared_error: 11821976.0000 - mean_absolute_error: 2759.4377
Epoch 00609: val_loss improved from 15964164.80519 to 15952778.38961, saving model to ./model_r_weights.h5
616/616 [==============================] - 0s 65us/sample - loss: 15941894.3896 - mean_squared_error: 15941893.0000 - mean_absolute_error: 3195.0127 - val_loss: 15952778.3896 - val_mean_squared_error: 15952778.0000 - val_mean_absolute_error: 3148.7468
Epoch 610/2000
 32/616 [>.............................] - ETA: 0s - loss: 16482471.0000 - mean_squared_error: 16482471.0000 - mean_absolute_error: 3314.2773
Epoch 00610: val_loss improved from 15952778.38961 to 15944430.38961, saving model to ./model_r_weights.h5
616/616 [==============================] - 0s 71us/sample - loss: 15931598.3117 - mean_squared_error: 15931598.0000 - mean_absolute_error: 3193.1682 - val_loss: 15944430.3896 - val_mean_squared_error: 15944430.0000 - val

Epoch 625/2000
 32/616 [>.............................] - ETA: 0s - loss: 24453652.0000 - mean_squared_error: 24453652.0000 - mean_absolute_error: 3840.2148
Epoch 00625: val_loss improved from 15848612.44156 to 15843756.77922, saving model to ./model_r_weights.h5
616/616 [==============================] - 0s 71us/sample - loss: 15807977.5065 - mean_squared_error: 15807977.0000 - mean_absolute_error: 3167.5337 - val_loss: 15843756.7792 - val_mean_squared_error: 15843757.0000 - val_mean_absolute_error: 3122.7710
Epoch 626/2000
 32/616 [>.............................] - ETA: 0s - loss: 14557517.0000 - mean_squared_error: 14557517.0000 - mean_absolute_error: 2954.2349
Epoch 00626: val_loss improved from 15843756.77922 to 15835690.93506, saving model to ./model_r_weights.h5
616/616 [==============================] - 0s 71us/sample - loss: 15803499.6883 - mean_squared_error: 15803500.0000 - mean_absolute_error: 3166.8718 - val_loss: 15835690.9351 - val_mean_squared_error: 15835691.0000 - val

Epoch 658/2000
 32/616 [>.............................] - ETA: 0s - loss: 16097198.0000 - mean_squared_error: 16097198.0000 - mean_absolute_error: 3381.9670
Epoch 00658: val_loss improved from 15727845.36364 to 15723923.11688, saving model to ./model_r_weights.h5
616/616 [==============================] - 0s 71us/sample - loss: 15636535.5974 - mean_squared_error: 15636535.0000 - mean_absolute_error: 3121.8433 - val_loss: 15723923.1169 - val_mean_squared_error: 15723922.0000 - val_mean_absolute_error: 3084.2043
Epoch 659/2000
 32/616 [>.............................] - ETA: 0s - loss: 16236458.0000 - mean_squared_error: 16236458.0000 - mean_absolute_error: 2974.6272
Epoch 00659: val_loss did not improve from 15723923.11688
616/616 [==============================] - 0s 45us/sample - loss: 15633513.2987 - mean_squared_error: 15633513.0000 - mean_absolute_error: 3120.6016 - val_loss: 15725496.9481 - val_mean_squared_error: 15725497.0000 - val_mean_absolute_error: 3083.4990
Epoch 660/2000
 3

Epoch 675/2000
 32/616 [>.............................] - ETA: 0s - loss: 17584244.0000 - mean_squared_error: 17584244.0000 - mean_absolute_error: 3221.1746
Epoch 00675: val_loss improved from 15694179.16883 to 15690821.32468, saving model to ./model_r_weights.h5
616/616 [==============================] - 0s 68us/sample - loss: 15583903.1688 - mean_squared_error: 15583904.0000 - mean_absolute_error: 3103.2637 - val_loss: 15690821.3247 - val_mean_squared_error: 15690822.0000 - val_mean_absolute_error: 3069.8240
Epoch 676/2000
 32/616 [>.............................] - ETA: 0s - loss: 11553792.0000 - mean_squared_error: 11553792.0000 - mean_absolute_error: 2527.2515
Epoch 00676: val_loss did not improve from 15690821.32468
616/616 [==============================] - 0s 54us/sample - loss: 15582947.8442 - mean_squared_error: 15582948.0000 - mean_absolute_error: 3102.9443 - val_loss: 15692006.4416 - val_mean_squared_error: 15692005.0000 - val_mean_absolute_error: 3069.2764
Epoch 677/2000
 3

Epoch 692/2000
 32/616 [>.............................] - ETA: 0s - loss: 19563440.0000 - mean_squared_error: 19563440.0000 - mean_absolute_error: 2989.4258
Epoch 00692: val_loss did not improve from 15658249.92208
616/616 [==============================] - 0s 45us/sample - loss: 15548290.1818 - mean_squared_error: 15548288.0000 - mean_absolute_error: 3087.1890 - val_loss: 15685484.5974 - val_mean_squared_error: 15685484.0000 - val_mean_absolute_error: 3058.6904


### 6. 모델 추론(validation)

In [20]:
predict_svm = model_svm.predict(testData_features_normed)
predict_dt  = model_dt.predict(testData_features_normed)
predict_kn  = model_kn.predict(testData_features_normed)
predict_rf  = model_rf.predict(testData_features_normed)

In [21]:
predictDf_svm = pd.DataFrame(predict_svm, columns=["predict_svc"])
predictDf_dt = pd.DataFrame(predict_dt, columns=["predict_dt"])
predictDf_kn = pd.DataFrame(predict_kn, columns=["predict_kn"])
predictDf_rf = pd.DataFrame(predict_rf, columns=["predict_rf"])

In [22]:
predictDf_dnn = pd.DataFrame(model.predict(testData_features_normed), columns=["predict_dnn"])

W1015 17:51:13.772546 12784 training.py:504] Falling back from v2 loop because of error: Failed to find data adapter that can handle input: <class 'pandas.core.frame.DataFrame'>, <class 'NoneType'>


In [23]:
model.evaluate(x=testData_features_normed,
              y=testData_label)

##nored 전 : [233259838.34196892, 233259860.0, 12164.15]
##nored 후 : [13376948.481865285, 13376950.0, 2895.6824]


W1015 17:51:13.817425 12784 training.py:504] Falling back from v2 loop because of error: Failed to find data adapter that can handle input: <class 'pandas.core.frame.DataFrame'>, <class 'NoneType'>


193/193 [==============================] - 0s 26us/sample - loss: 13768860.2902 - mean_squared_error: 13768859.0000 - mean_absolute_error: 2973.3145


[13768860.29015544, 13768859.0, 2973.3145]

In [24]:
lose, mse, mae = model.evaluate(x=testData_features_normed,
              y=testData_label)

W1015 17:51:13.833382 12784 training.py:504] Falling back from v2 loop because of error: Failed to find data adapter that can handle input: <class 'pandas.core.frame.DataFrame'>, <class 'NoneType'>


193/193 [==============================] - 0s 26us/sample - loss: 13768860.2902 - mean_squared_error: 13768859.0000 - mean_absolute_error: 2973.3145


In [25]:
len(predictDf_dnn)

193

In [26]:
testData_freatures2 = testData_features.reset_index(drop=True)
testData_label2 = testData_label.reset_index(drop=True)

In [27]:
finalData = pd.concat([testData_freatures2,testData_label2,predictDf_dnn,predictDf_svm,predictDf_dt,predictDf_kn,predictDf_rf],axis=1)
finalData.head(10)

,miles,debt,income,sales,predict_dnn,predict_svc,predict_dt,predict_kn,predict_rf
0,20,414,9900,5265,9765.487305,17677,5029,1376,6041
1,32,51908,7169,23894,24398.501953,25125,25118,22508,25715
2,32,9503,5479,9319,10406.263672,7596,9533,6042,8526
3,23,5809,5548,13722,8424.601562,7596,13790,1285,13790
4,29,15752,6277,22145,12572.067383,7596,12256,6042,6626
5,12,903,6314,2440,6397.094238,3510,4101,2228,3779
6,30,0,1673,2182,4491.422363,2869,1789,763,1789
7,35,4339,7852,6025,10947.153320,17677,6250,6240,16678
8,25,4133,4212,4376,7124.098145,4609,1836,1492,3978
9,25,5844,5376,12748,8511.530273,7596,6376,1285,6376


In [28]:
finalData.dtypes

miles            int64
debt             int64
income           int64
sales            int64
predict_dnn    float32
predict_svc      int64
predict_dt       int64
predict_kn       int64
predict_rf       int64
dtype: object

In [29]:
finalData["predictDf_dnn"] = finalData.predict_dnn.astype(int)

In [30]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

In [35]:
#결과 테스트하기
ac_score_dnn = mean_absolute_error(finalData.sales, finalData.predict_dnn)
ac_score_svc = mean_absolute_error(finalData.sales, finalData.predict_svc)
ac_score_dt = mean_absolute_error(finalData.sales, finalData.predict_dt)
ac_score_kn = mean_absolute_error(finalData.sales, finalData.predict_kn)
ac_score_rf = mean_absolute_error(finalData.sales, finalData.predict_rf)

sq_score_dnn = mean_squared_error(finalData.sales, finalData.predict_dnn)
sq_score_svc = mean_squared_error(finalData.sales, finalData.predict_svc)
sq_score_dt = mean_squared_error(finalData.sales, finalData.predict_dt)
sq_score_kn = mean_squared_error(finalData.sales, finalData.predict_kn)
sq_score_rf = mean_squared_error(finalData.sales, finalData.predict_rf)

##결과 리포트하기
print("mean_absolute_error")
print("Accuracy dnn =", ac_score_dnn)
print("Accuracy svm =", ac_score_svc)
print("Accuracy dt=", ac_score_dt)
print("Accuracy kn=", ac_score_kn)
print("Accuracy rf=", ac_score_rf)
print("mean_squared_error")
##결과 리포트하기
print("Accuracy dnn =", sq_score_dnn)
print("Accuracy svm =", sq_score_svc)
print("Accuracy dt=", sq_score_dt)
print("Accuracy kn=", sq_score_kn)
print("Accuracy rf=", sq_score_rf)

mean_absolute_error
Accuracy dnn = 2973.3144069533273
Accuracy svm = 3676.958549222798
Accuracy dt= 2848.1502590673576
Accuracy kn= 4532.430051813471
Accuracy rf= 3395.481865284974
mean_squared_error
Accuracy dnn = 13768860.613422725
Accuracy svm = 28820614.305699483
Accuracy dt= 19305661.476683937
Accuracy kn= 38786422.33678757
Accuracy rf= 26474986.196891192
